In [1]:
import cv2
import numpy as np
import logging
import os
import time
from collections import deque

# Configura il logging con un file di log e un output a schermo
def setup_logging(output_dir):
    log_file = os.path.join(output_dir, "processing.log")
    
    # Rimuovi tutti i vecchi handler, se esistono
    root_logger = logging.getLogger()
    while root_logger.hasHandlers():
        root_logger.removeHandler(root_logger.handlers[0])
    
    # Configura il logging con il flush immediato
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - %(message)s",
        handlers=[
            logging.FileHandler(log_file, mode='w', delay=False),  # Scrive subito
            logging.StreamHandler()
        ]
    )
    logging.info(f"Logging configurato. File di log salvato in: {log_file}")

# Calcola il flusso ottico tra frame successivi di un video (senza riduzione di risoluzione),
def temporal_smoothing_flow(
    video_path,
    output_dir,
    flow_threshold=0.5,
    alpha_fraction=0.2,
    window_size=30,
    morph_kernel=2,
    save_name="overlay.mp4",
    mask_save_name="mask.mp4"
):
    """
    Calcola il flusso ottico tra frame successivi di un video (senza riduzione di risoluzione),
    accumula le aree di movimento in una coda (window_size), effettua operazioni morfologiche
    di apertura/chiusura e salva:
      - Un video 'overlay.mp4' che è la copia del video originale.
      - Un video 'mask.mp4' con la maschera di movimento rettangolarizzata.
    """
    start_time = time.time()
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        logging.error(f"Impossibile aprire il video in ingresso: {video_path}")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    overlay_path = os.path.join(output_dir, save_name)
    mask_path = os.path.join(output_dir, mask_save_name)

    # Impostiamo il codec video: 'mp4v' è un codec solitamente ben supportato.
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_overlay = cv2.VideoWriter(overlay_path, fourcc, fps, (width, height))
    out_mask = cv2.VideoWriter(mask_path, fourcc, fps, (width, height), isColor=False)

    ret, first_frame = cap.read()
    if not ret:
        logging.error("Impossibile leggere il primo frame del video.")
        cap.release()
        return

    # Convertiamo il primo frame in scala di grigi alla risoluzione piena
    prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)

    mask_queue = deque(maxlen=window_size)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (morph_kernel, morph_kernel))

    frame_count = 0  # Per tracciare a che frame siamo

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calcolo del flusso ottico Farneback su tutta la risoluzione originale
        flow = cv2.calcOpticalFlowFarneback(
            prev_gray, gray, None, 
            0.3,   # pyr_scale
            2,     # levels
            9,     # winsize
            2,     # iterations
            5,     # poly_n
            1.1,   # poly_sigma
            0      # flags
        )
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1], angleInDegrees=False)

        # Maschera con soglia su mag
        mask_current = (mag > flow_threshold).astype(np.uint8) * 255
        mask_queue.append(mask_current)

        # Somma le maschere nella coda
        cumulative_mask = np.sum(np.array(mask_queue), axis=0)

        # Soglia in base ad alpha_fraction
        mask_smoothed = (cumulative_mask >= (alpha_fraction * len(mask_queue) * 255)).astype(np.uint8) * 255

        # Operazioni morfologiche di apertura/chiusura
        mask_smoothed = cv2.morphologyEx(mask_smoothed, cv2.MORPH_CLOSE, kernel)
        mask_smoothed = cv2.morphologyEx(mask_smoothed, cv2.MORPH_OPEN, kernel)

        # Retangolarizzazione delle aree di movimento
        contours, _ = cv2.findContours(mask_smoothed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        mask_rect = np.zeros((height, width), dtype=np.uint8)

        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            x_padded = max(0, x )
            y_padded = max(0, y )
            w_padded = min(w , width - x_padded)
            h_padded = min(h  , height - y_padded)
            cv2.rectangle(mask_rect, (x_padded, y_padded),
                          (x_padded + w_padded, y_padded + h_padded), 255, -1)

        # Scrittura dei frame nei rispettivi video
        out_overlay.write(frame)
        out_mask.write(mask_rect)

        prev_gray = gray.copy()

    cap.release()
    out_overlay.release()
    out_mask.release()
    end_time = time.time()
    logging.info(
        f"Temporal smoothing flow completed for '{os.path.basename(video_path)}' "
        f"in {end_time - start_time:.2f} seconds. Totale frame elaborati: {frame_count}"
    )

# Comprime un video basandosi sul movimento rilevato dalla maschera
def compress_with_motion(input_video, mask_video, output_dir):
    """
    Esegue la compressione basata sul movimento (DCT + quantizzazione) usando OpenCV:
    - Sulle aree di movimento (indicate dalla maschera) lascia i frame originali a colori.
    - Sulle aree senza movimento, applica una compressione DCT più aggressiva e in più
      converte quella zona in bianco e nero (scala di grigi).
    """
    start_time = time.time()
    logging.info(f"Avvio della motion-based compression per: {os.path.basename(input_video)}")

    cap_input = cv2.VideoCapture(input_video)
    cap_mask = cv2.VideoCapture(mask_video)

    if not cap_input.isOpened():
        logging.error(f"Impossibile aprire il video originale: {input_video}")
        return
    if not cap_mask.isOpened():
        logging.error(f"Impossibile aprire il video maschera: {mask_video}")
        return

    fps = cap_input.get(cv2.CAP_PROP_FPS)
    width = int(cap_input.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap_input.get(cv2.CAP_PROP_FRAME_HEIGHT))

    output_video = os.path.join(output_dir, "compressed.mp4")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

    # Matrice di quantizzazione (più alto => più compressione)
    QTY_aggressive = np.full((8, 8), 100, dtype=np.float32)

    frame_count = 0

    while True:
        ret_in, frame_in = cap_input.read()
        ret_mask, frame_mask = cap_mask.read()
        if not ret_in or not ret_mask:
            break
        frame_count += 1

        # Convertiamo la maschera in scala di grigi se necessario
        if len(frame_mask.shape) == 3:
            frame_mask = cv2.cvtColor(frame_mask, cv2.COLOR_BGR2GRAY)

        # Convertiamo il frame in YCrCb per applicare la DCT solo sul canale di luminanza
        frame_ycrcb = cv2.cvtColor(frame_in, cv2.COLOR_BGR2YCrCb)
        channels = cv2.split(frame_ycrcb)

        # Scorriamo a blocchi 8x8
        for i in range(0, frame_mask.shape[0], 8):
            for j in range(0, frame_mask.shape[1], 8):
                # Se in questa zona non c'è movimento, applichiamo DCT + quantizzazione
                # e successivamente convertiamo la zona in B/N.
                if frame_mask[i:i+8, j:j+8].mean() == 0:
                    for c in range(3):
                        block = channels[c][i:i+8, j:j+8]
                        if block.shape == (8, 8):
                            dct_block = cv2.dct(block.astype(np.float32) - 128)
                            quantized_block = np.round(dct_block / QTY_aggressive) * QTY_aggressive
                            idct_block = cv2.idct(quantized_block) + 128
                            channels[c][i:i+8, j:j+8] = np.clip(idct_block, 0, 255)

        # Ricostruiamo il frame compresso in BGR
        frame_processed = cv2.merge(channels)
        frame_processed = cv2.cvtColor(frame_processed, cv2.COLOR_YCrCb2BGR)

        # Convertiamo in B/N solo i blocchi senza movimento
        for i in range(0, frame_mask.shape[0], 8):
            for j in range(0, frame_mask.shape[1], 8):
                if frame_mask[i:i+8, j:j+8].mean() == 0:
                    roi = frame_processed[i:i+8, j:j+8]
                    gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
                    gray_roi_bgr = cv2.cvtColor(gray_roi, cv2.COLOR_GRAY2BGR)
                    frame_processed[i:i+8, j:j+8] = gray_roi_bgr

        out.write(frame_processed)

    cap_input.release()
    cap_mask.release()
    out.release()
    end_time = time.time()
    logging.info(
        f"Motion-based compression completed for '{os.path.basename(input_video)}' "
        f"in {end_time - start_time:.2f} seconds. Totale frame elaborati: {frame_count}"
    )


def main():
    """
    Esegue l'intero flusso:
    - Per ogni file .mp4 presente nella cartella di input,
      1. Crea una sottocartella di output dedicata (es. output/<nome_file_senza_estensione>)
      2. Imposta il logging in quella cartella
      3. Esegue:
         a) temporal_smoothing_flow --> genera overlay.mp4 e mask.mp4
         b) compress_with_motion   --> genera compressed.mp4
    """
    start_time_global = time.time()
    input_dir = "../Dataset/input/"
    output_dir = "../Dataset/output/"
    
    # Trova tutti i file mp4 presenti in input_dir
    video_list = [f for f in os.listdir(input_dir) if f.lower().endswith(".mp4")]
    if not video_list:
        print("Nessun file mp4 trovato nella cartella di input.")
        return
    
    # Log di riepilogo
    logging.info("=== INIZIO ELABORAZIONE DI TUTTI I VIDEO ===")
    logging.info(f"Video trovati: {video_list}")

    # Processa ciascun video
    for input_video in video_list:
        video_start_time = time.time()

        video_name = os.path.splitext(input_video)[0]
        video_output_dir = os.path.join(output_dir, video_name)

        # Crea la cartella di output per il singolo video
        if not os.path.exists(video_output_dir):
            os.makedirs(video_output_dir)

        # 1. Setup logging per il singolo video
        setup_logging(video_output_dir)
        logging.info(f"=== Inizio elaborazione del video '{input_video}' ===")

        # 2. Calcolo del flusso ottico e generazione overlay + mask
        logging.info(f"Eseguo temporal_smoothing_flow su '{input_video}'...")
        temporal_smoothing_flow(
            video_path=os.path.join(input_dir, input_video),
            output_dir=video_output_dir,
            flow_threshold=0.5,    # Parametri di default (regolabili a piacere)
            alpha_fraction=0.2,
            window_size=30,
            morph_kernel=2,
            save_name=f"{video_name}_overlay.mp4",
            mask_save_name=f"{video_name}_mask.mp4"
        )
        logging.info(f"temporal_smoothing_flow terminato per '{input_video}'.")

        # 3. Compressione basata sul movimento + conversione in B/N delle zone statiche
        logging.info(f"Eseguo compress_with_motion su '{video_name}_overlay.mp4' e '{video_name}_mask.mp4'...")
        compress_with_motion(
            input_video=os.path.join(video_output_dir, f"{video_name}_overlay.mp4"),
            mask_video=os.path.join(video_output_dir, f"{video_name}_mask.mp4"),
            output_dir=video_output_dir
        )
        logging.info(f"compress_with_motion terminato per '{input_video}'.")

        video_end_time = time.time()
        logging.info(
            f"=== Elaborazione video '{video_name}' completata in {video_end_time - video_start_time:.2f} secondi. ===\n"
        )

    end_time_global = time.time()
    logging.info(f"=== Elaborazione di tutti i video completata in {end_time_global - start_time_global:.2f} secondi. ===")


if __name__ == "__main__":
    main()

2025-01-22 18:31:08,363 - INFO - Logging configurato. File di log salvato in: ../Dataset/output/test1/processing.log
2025-01-22 18:31:08,363 - INFO - === Inizio elaborazione del video 'test1.mp4' ===
2025-01-22 18:31:08,363 - INFO - Eseguo temporal_smoothing_flow su 'test1.mp4'...
OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
2025-01-22 18:31:31,323 - INFO - Temporal smoothing flow completed for 'test1.mp4' in 22.96 seconds. Totale frame elaborati: 125
2025-01-22 18:31:31,330 - INFO - temporal_smoothing_flow terminato per 'test1.mp4'.
2025-01-22 18:31:31,330 - INFO - Eseguo compress_with_motion su 'test1_overlay.mp4' e 'test1_mask.mp4'...
2025-01-22 18:31:31,331 - INFO - Avvio della motion-based compression